In [ ]:
%pylab inline

This snippet of code adds the source code to the PYTHONPATH.  It assumes that the notebook and source directories are under the same parent directory.  If the directory structure has been changed then os.path.join(sys.path[0],'..','source') will need to be replaced with the new path to source.  This isn't a very clean solution but it is the only one I can find that will work.

In [ ]:
import sys
import os
sys.path.append(os.path.join(sys.path[0],'..','source'))

In [ ]:
def get_barres_files():
    
    import os, glob
    diri = r"C:/Users/apbarret/Documents/data/Spectra/*/archive"
    return glob.glob(os.path.join(diri,'*_spec.hdr'))
    
def plot_from_file(fn, ax, color='k', lw=2):
    
    import spectral.io.envi as envi
    import matplotlib.pyplot as plt
    
    spectra = envi.open(fn)
    
    ax.plot(spectra.bands.centers, moving_average(spectra.spectra.reshape(-1),11), 
            lw=lw, color=color)
    
def moving_average(x, window_len):
    import numpy as np
    
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    w = np.ones(window_len,'d')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len-1)*0.5:(-1)*(window_len-1)*0.5]

## Read spectral libraries

In [ ]:
from spectral.io import envi
from constants import ICE_LIBRARY_PATH, POND_LIBRARY_PATH, OWTR_LIBRARY_PATH

# Ice
ice = envi.open(ICE_LIBRARY_PATH)
# Ponds
pond = envi.open(POND_LIBRARY_PATH)
# Leads
lead = envi.open(OWTR_LIBRARY_PATH)

# Get list of Barrow and resolute files
fList = get_barres_files()

## Get indices for dark, light and ponds with ice

In [ ]:
import re
import numpy as np
isdark = re.compile('dark', re.IGNORECASE)
dark_idx = np.array([i for i, tgt in enumerate(pond.names) if isdark.search(tgt)])

islight = re.compile('light', re.IGNORECASE)
light_idx = np.array([i for i, tgt in enumerate(pond.names) if islight.search(tgt)])

isice = re.compile('brash|frazil', re.IGNORECASE)
ice_idx = np.array([i for i, tgt in enumerate(pond.names) if isice.search(tgt)])

isother = re.compile('^dark|^light|^brash|^frazil', re.IGNORECASE)
other_idx = np.array([i for i, tgt in enumerate(pond.names) if isother.search(tgt)])


In [ ]:
lead.bands.centers.__sizeof__()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shebaspectral import add_modis_bands

xmin = 300.
xmax = 1400.

f, ax = plt.subplots(1,3, figsize=(15,7))

ax[0].plot(ice.bands.centers, ice.spectra.T, 'k', lw=2)
# Barrow and Resolute spectra:
# Snow
plot_from_file(fList[0], ax[0], color='red')
plot_from_file(fList[7], ax[0], color='orange') # Glazed snow
plot_from_file(fList[9], ax[0], color='red')
# Ice
plot_from_file(fList[1], ax[0], color='cyan') # Melting ice
plot_from_file(fList[10], ax[0], color='blue') # Blue ice
plot_from_file(fList[2], ax[0], color='k')
plot_from_file(fList[8], ax[0], color='k')

ax[0].set_xlim(xmin,xmax)
ax[0].set_ylim(0.,1.)
ax[0].set_title("Ice and Snow", fontsize=20)
ax[0].set_ylabel("Spectral Reflectance", fontsize=18)
ax[0].set_xlabel("Wavelength [nm]", fontsize=18)
add_modis_bands(ax=ax[0])

# Set up legend for pond plot
snow_line = mlines.Line2D([], [], color='red', lw=2,
                          label='Snow')
glazedSnow_line = mlines.Line2D([], [], color='orange', lw=2,
                          label='Glazed Snow')
ice_line = mlines.Line2D([], [], color='k', lw=2,
                          label='White Ice')
meltingIce_line = mlines.Line2D([], [], color='cyan', lw=2,
                          label='Melting Ice')
blueIce_line = mlines.Line2D([], [], color='blue', lw=2,
                          label='Blue Ice')
ax[0].legend(handles=[snow_line, glazedSnow_line, ice_line, meltingIce_line, blueIce_line], loc=3)

ax[1].plot(pond.bands.centers, pond.spectra[dark_idx,:].T, 'k', lw=2, label="Dark")
ax[1].plot(pond.bands.centers, pond.spectra[light_idx,:].T, 'r', lw=2, label="light")
ax[1].plot(pond.bands.centers, pond.spectra[ice_idx,:].T, 'b', lw=2, label="ice")
# Add pond spectra from barrow and resolute
plot_from_file(fList[4], ax[1], color='k')
plot_from_file(fList[5], ax[1], color='k')
plot_from_file(fList[6], ax[1], color='k')

ax[1].set_xlim(xmin,xmax)
ax[1].set_ylim(0.,1.)
ax[1].set_title("Melt Pond", fontsize=20)
add_modis_bands(ax=ax[1])
ax[1].set_xlabel("Wavelength [nm]", fontsize=18)

# Set up legend for pond plot
darkpond_line = mlines.Line2D([], [], color='k', lw=2,
                          label='Dark Ponds')
lightpond_line = mlines.Line2D([], [], color='r', lw=2,
                          label='Light Ponds')
icepond_line = mlines.Line2D([], [], color='b', lw=2,
                          label='Ponds with ice')
ax[1].legend(handles=[darkpond_line, lightpond_line, icepond_line])

ax[2].plot(lead.bands.centers, lead.spectra[0,:].T, 'k', lw=2, label='Open Lead')
ax[2].plot(lead.bands.centers, lead.spectra[1:,:].T, 'b', lw=2, label='Leads with ice')
# Add pond spectra from barrow and resolute
plot_from_file(fList[3], ax[2], color='k')

ax[2].set_xlim(xmin,xmax)
ax[2].set_ylim(0.,1.)
ax[2].set_title("Lead", fontsize=20)
add_modis_bands(ax=ax[2])
ax[2].set_xlabel("Wavelength [nm]", fontsize=18)

openlead_line = mlines.Line2D([], [], color='k', lw=2,
                          label='Open Lead')
icelead_line = mlines.Line2D([], [], color='b', lw=2,
                          label='Leads with ice')
ax[2].legend(handles=[openlead_line,icelead_line])

plotdiri = r"C:/Users/apbarret/Documents/data/Spectra"
plotfile = 'sheba_barrow_resolute_spectra.png'
f.savefig(os.path.join(plotdiri,plotfile))